In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)

from trl import SFTTrainer

2023-08-08 12:50:53.546413: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-08 12:50:53.646401: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-08 12:50:54.136913: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-08 12:50:54.136964: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [2]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf', trust_remote_code=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"
max_seq_length = 1024

In [3]:
tokenizer.eos_token

'</s>'

In [4]:
tokenizer.pad_token_id

0

In [5]:
!head -n 1000 shuf-combine-1024.jsonl > example.jsonl

In [6]:
from trl.trainer.utils import ConstantLengthDataset
from datasets import load_dataset
import random

In [7]:
tokenizer.eos_token

'</s>'

In [8]:
def generate_and_tokenize_prompt(row):
    text = f"#User: {row['input']}\n#Bot: {row['output']}{tokenizer.eos_token}"
    return {'text': text}


dataset = load_dataset("json", data_files='example.jsonl', split="train")
dataset = dataset.map(generate_and_tokenize_prompt)
dataset = dataset.remove_columns(['prompt_input', 'input', 'output'])

Using custom data configuration default-e3ae29104c133a97


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /home/husein/.cache/huggingface/datasets/json/default-e3ae29104c133a97/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1000 [00:00<?, ?ex/s]

In [9]:
dataset['text'][0]

'#User: text `Hello &amp; Good morning, Twitteriansz Nak bagitahu ni. Dekat kedai Lising or senang kata, depan Sk Satria Jaya, ada`, categorize the text with label [\'negative\', \'positive\', \'neutral\'] and explain, return as JSON key {\'sentiment\', \'explain_en\', \'explain_ms\'}\n#Bot: {\n    "sentiment": "neutral",\n    "explain_en": "The text is neutral as it does not express any strong positive or negative emotion. It is simply providing information about a location.",\n    "explain_ms": "Teks ini adalah neutral kerana ia tidak mengungkapkan emosi positif atau negatif yang kuat. Ia hanya memberikan maklumat tentang lokasi."\n}</s>'

In [10]:
# trainer = SFTTrainer(
#     model=model,
#     train_dataset=dataset,
#     peft_config=peft_config,
#     dataset_text_field="text",
#     max_seq_length=script_args.max_seq_length,
#     tokenizer=tokenizer,
#     args=training_arguments,
#     packing=script_args.packing,
# )

In [11]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [12]:
dataset_text_field = 'text'
use_formatting_func = None
max_seq_len = max_seq_length
def tokenize(element):
    outputs = tokenizer(
        element[dataset_text_field] if not use_formatting_func else formatting_func(element),
        truncation=True,
        padding=False,
        max_length=max_seq_len,
        return_overflowing_tokens=False,
        return_length=False,
    )

    if use_formatting_func and not self._dataset_sanity_checked:
        if not isinstance(formatting_func(element), list):
            raise ValueError(
                "The `formatting_func` should return a list of processed strings since it can lead to silent bugs."
            )
        else:
            self._dataset_sanity_checked = True

    return {"input_ids": outputs["input_ids"], "attention_mask": outputs["attention_mask"]}

In [13]:
dataset.column_names

['text']

In [14]:
tokenized_dataset = dataset.map(
    tokenize,
    batched=True,
    remove_columns=dataset.column_names,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
tokenizer.decode(tokenized_dataset['input_ids'][0])

'<s> #User: text `Hello &amp; Good morning, Twitteriansz Nak bagitahu ni. Dekat kedai Lising or senang kata, depan Sk Satria Jaya, ada`, categorize the text with label [\'negative\', \'positive\', \'neutral\'] and explain, return as JSON key {\'sentiment\', \'explain_en\', \'explain_ms\'}\n#Bot: {\n    "sentiment": "neutral",\n    "explain_en": "The text is neutral as it does not express any strong positive or negative emotion. It is simply providing information about a location.",\n    "explain_ms": "Teks ini adalah neutral kerana ia tidak mengungkapkan emosi positif atau negatif yang kuat. Ia hanya memberikan maklumat tentang lokasi."\n}</s>'

In [16]:
b = [tokenized_dataset[i] for i in range(3)]

In [17]:
b = data_collator(b)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [18]:
b['input_ids'][0]

tensor([    1,   396,  2659, 29901,  1426,   421, 10994,   669,  1160, 29936,
         7197,  7250, 29892, 20147,  5834, 29920, 20962, 19548,   277, 29017,
         6836, 29889,   897, 29895,   271,   413,   287,  1794,   365,  5921,
          470,  6940,   574,   413,   532, 29892,  1401,   273,  4971, 12178,
         2849,   435,  9010, 29892,   594, 29874,  1673, 11608,   675,   278,
         1426,   411,  3858,  6024, 22198,   742,   525,  1066,  3321,   742,
          525, 17821,  1705,  2033,   322,  5649, 29892,   736,   408,  4663,
         1820, 11117, 18616,  2073,   742,   525,  4548,  7420, 29918,   264,
          742,   525,  4548,  7420, 29918,  1516, 10827,    13, 29937, 29933,
          327, 29901,   426,    13,  1678,   376, 18616,  2073,  1115,   376,
        17821,  1705,   613,    13,  1678,   376,  4548,  7420, 29918,   264,
         1115,   376,  1576,  1426,   338, 21104,   408,   372,   947,   451,
         4653,   738,  4549,  6374,   470,  8178,   953,  8194, 

In [19]:
b['labels'][0]

tensor([    1,   396,  2659, 29901,  1426,   421, 10994,   669,  1160, 29936,
         7197,  7250, 29892, 20147,  5834, 29920, 20962, 19548,   277, 29017,
         6836, 29889,   897, 29895,   271,   413,   287,  1794,   365,  5921,
          470,  6940,   574,   413,   532, 29892,  1401,   273,  4971, 12178,
         2849,   435,  9010, 29892,   594, 29874,  1673, 11608,   675,   278,
         1426,   411,  3858,  6024, 22198,   742,   525,  1066,  3321,   742,
          525, 17821,  1705,  2033,   322,  5649, 29892,   736,   408,  4663,
         1820, 11117, 18616,  2073,   742,   525,  4548,  7420, 29918,   264,
          742,   525,  4548,  7420, 29918,  1516, 10827,    13, 29937, 29933,
          327, 29901,   426,    13,  1678,   376, 18616,  2073,  1115,   376,
        17821,  1705,   613,    13,  1678,   376,  4548,  7420, 29918,   264,
         1115,   376,  1576,  1426,   338, 21104,   408,   372,   947,   451,
         4653,   738,  4549,  6374,   470,  8178,   953,  8194, 